In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Cargar datos
costos_df = pd.read_excel('Costos_Subprocesos.xlsx')
indicadores_df = pd.read_excel('Indicadores_Minas.xlsx')

# Convertir las fechas a formato datetime
costos_df['Subproceso_Costo'] = pd.to_datetime(costos_df['Subproceso_Costo'])
indicadores_df['Fecha'] = pd.to_datetime(indicadores_df['Fecha'])

# Unir los DataFrames por la columna de fechas
df_unificado = pd.merge(costos_df, indicadores_df, left_on='Subproceso_Costo', right_on='Fecha', how='inner')
df_unificado = df_unificado.drop(columns=['Fecha'])

# Definir subprocesos e indicadores
subprocesos_cols = costos_df.columns[1:]  # Excluyendo la columna de fecha
indicadores_cols = indicadores_df.columns[1:]  # Excluyendo la columna de fecha

# Calcular diferentes tipos de correlación
correlation_methods = {
    'Pearson': df_unificado.corr(method='pearson'),
    'Spearman': df_unificado.corr(method='spearman'),
    'Kendall': df_unificado.corr(method='kendall')
}

# Resultados para cada método
resultados_comparacion = {}

for method, correlation_matrix in correlation_methods.items():
    correlation_filtered = correlation_matrix.loc[subprocesos_cols, indicadores_cols]
    top_correlations = correlation_filtered.apply(lambda x: x.nlargest(3), axis=1)

    # Dividir los datos en entrenamiento (2016-2021) y prueba (2022)
    train_data = df_unificado[df_unificado['Subproceso_Costo'] < '2022-01-01']
    test_data = df_unificado[df_unificado['Subproceso_Costo'] == '2022-01-01']

    # Modelo Neural Network (Redes Neuronales)
    resultados_nn = {}
    nn_model = MLPRegressor(hidden_layer_sizes=(100,), max_iter=100000, random_state=42)
    
    for subproceso in subprocesos_cols:
        indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
        if len(indicadores_seleccionados) > 0:
            X_train = train_data[indicadores_seleccionados]
            y_train = train_data[subproceso]
            X_test = test_data[indicadores_seleccionados]
            nn_model.fit(X_train, y_train)
            y_pred_2022 = nn_model.predict(X_test)[0]
            costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-01-01'][subproceso].values[0]
            diferencia = y_pred_2022 - costo_real_2022
            diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
            resultados_nn[subproceso] = {
                'Predicción 2022': y_pred_2022, 
                'Costo Real 2022': costo_real_2022, 
                'Diferencia': diferencia, 
                'Diferencia %': diferencia_pct
            }
    
    # Guardar los resultados para este método de correlación
    resultados_comparacion[method] = pd.DataFrame(resultados_nn).T

# Comparar y seleccionar la mejor correlación para cada subproceso
mejor_correlacion = pd.DataFrame(columns=['Subproceso', 'Mejor Correlación', 'Diferencia %'])

for subproceso in subprocesos_cols:
    mejor_diferencia_pct = float('inf')
    mejor_metodo = None
    
    for method, resultados_df in resultados_comparacion.items():
        if subproceso in resultados_df.index:
            diferencia_pct = resultados_df.loc[subproceso, 'Diferencia %']
            if isinstance(diferencia_pct, (int, float)) and abs(diferencia_pct) < abs(mejor_diferencia_pct):
                mejor_diferencia_pct = diferencia_pct
                mejor_metodo = method
    
    mejor_correlacion = mejor_correlacion.append({
        'Subproceso': subproceso,
        'Mejor Correlación': mejor_metodo,
        'Diferencia %': mejor_diferencia_pct
    }, ignore_index=True)

# Mostrar los resultados
print("Comparación de métodos de correlación:")
print(mejor_correlacion)


C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maxi